In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np

import spacy
import nltk
import nltk.data
from nltk.tokenize import word_tokenize
import regex as re
import string
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [3]:
df = pd.read_csv('../data/input_tweets.csv')
print(df.head())

   Unnamed: 0         username  \
0           0  cristia89872894   
1           2      sccristiano   
2          12      Atorrante15   
3          16          _fedenb   
4          20  javier_eduardo3   

                                                                                                                                         acctdesc  \
0  Hincha del Apruebo, pero nunca de los Apruebistas...mi izquierda llega sólo hasta Mujica. Me gustan los chistes sobre estupideces posmodernas.   
1                                                                                                                                             NaN   
2                                                                                                                                             NaN   
3                                                                                                                           🍃ρєя αsρєяα α∂ αsтяα💫   
4                                                 

In [4]:
df.drop(columns = ['Unnamed: 0', 'username', 'acctdesc' ,'location', 'following', 'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts', 'retweetcount', 'hashtags', ], inplace = True)
print(df.head(5))

                                                                                                                                                                                                                                                                      text  \
0                                                        @liznorthon @SoyAIvaro @ursulaeggers @MielyMerken @AlessandriFelip Pillaron al presunto, es de nacionalidad colombiana. No es sorpresa, la violencia es parte del inmigrante de origen colombiano, es su cultura.   
1                                                                                                                             @ccarvajaly Cuando se den cuenta que eran homicidas o narco y los pillen dirán hemos dado con este inmigrante ilegal que era un delincuente.   
2                                              @SergioChouza @santidemoreno11 Jajaajja ahí estuviste bien chouza se nota TU CABEZA DE SUDACA Y DE MAL PAGADOR,lo llevan en el ADN!!! esos pape

In [12]:
df_train, df_test = train_test_split(df, test_size = 0.3, random_state = 42)

print(f'Training set shape: {df_train.shape}')
print(f'Test set shape: {df_test.shape}')

target
0    1027
1     973
dtype: int64
target
0    720
1    680
dtype: int64
target
0    307
1    293
dtype: int64
Training set shape: (1400, 2)
Test set shape: (600, 2)


In [16]:
punctuations = "¡!#$%&'()*+,-./:;<=>¿?@[\]^_`{|}~"

def read_txt(filename):
    list = []
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.readlines()
        for line in data:
            list.append(str(line).replace('\n', ''))
    return list

stopwords = read_txt('../data/spanish_stopwords.txt')

['a', 'a', 'acá', 'ademas', 'además', 'ahí', 'ajena', 'al', 'algo', 'algún', 'allá', 'ambos', 'ante', 'antes', 'aquel', 'aquella', 'aquí', 'arriba', 'así', 'aun', 'cabe', 'cada', 'como', 'con', 'conmigo', 'conseguir', 'consigo', 'consigue', 'consiguen', 'consigues', 'contigo', 'cual', 'cuales', 'cualquier', 'cuando', 'cuanto', 'de', 'dejar', 'del', 'dentro', 'desde', 'donde', 'dos', 'e', 'el', 'él', 'ella', 'emplean', 'emplear', 'empleas', 'empleo', 'en', 'encima', 'entonces', 'entre', 'eramos', 'eran', 'eres', 'es', 'esa', 'eso', 'esta', 'estaba', 'estado', 'estamos', 'están', 'estar', 'este', 'estoy', 'etc', 'fin', 'fue', 'fueron', 'fui', 'fuimos', 'ha', 'hacéis', 'hacemos', 'hacen', 'hacer', 'hacia', 'hago', 'hasta', 'he', 'incluso', 'intenta', 'intentáis', 'intentamos', 'intentan', 'intentar', 'intento', 'ir', 'junto', 'la', 'las', 'le', 'lo', 'los', 'mas', 'más', 'me', 'mi', 'mía', 'mientras', 'mío', 'mis', 'misma', 'modo', 'nos', 'nosotras', 'nuestra', 'nunca', 'otra', 'para', 'p